In [9]:
import json
import elasticsearch
import pprint
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from functions.fad_crawl.helpers.esGenData import *
from functions.fad_crawl.helpers.processingData import *
from functions.fad_crawl.spiders.models.constants import ELASTICSEARCH_HOST

In [10]:
with open("functions/schemaData/financeInfo/Joint Stock Company_Mining, Quarrying, and Oil and Gas Extraction_PTK_CSTC_Quarter_Page_1.json", "r") as jsonfile:
    j=json.load(jsonfile)
    jsonfile.close()
with open("functions/schema/lookup_dict_all_nonfin.json", "r") as jsonfile:
    LOOKUP_DICT=json.load(jsonfile)
    jsonfile.close()
with open("functions/schema/mapping_dict_all_nonfin.json", "r") as jsonfile:
    MAPPING_DICT=json.load(jsonfile)
    jsonfile.close()

In [11]:
def output(report, resp_json, lookup_dict, mapping_dict):
    output = {}
    for period in resp_json[0]:
        period_name = period['TermNameEN'] + "-" + str(period['YearPeriod'])
        period_key, period_id = getKey(period)
        output[period_key] = {"ID" : period_id, "Period": period_name}
    for report_fullname in resp_json[1].keys():
        for k in output.keys():
            output[k][report_fullname] = {}
        for acc_content in resp_json[1][report_fullname]:
            try:
                acc_n = get_fad_acc(report, report_fullname, acc_content, lookup_dict, mapping_dict)
            except:
                acc_n = simplifyText(acc_content['Name']) + ";nonFAD"
            for item in output.items():
                item[1][report_fullname][acc_n] = acc_content["Value"+item[1]["ID"]]
    return output
        
        # Processing data to ES friendly format
#         output = processFinanceInfo(output, id)
#         index = "balancesheets"

In [12]:
def processFinanceInfoX(output, _id=""):
    output_ = []
    for timestamp, output_content in output.items():
        start, end = getDate(timestamp)
        ### Checking if all data is null then skip
        _ = True
        for content in output_content.values():
            if isinstance(content, dict):
                for acc_value in content.values():
                    if acc_value != None:
                        _ = False
                        break
        if not _:
            ### Handle when the key is empty
            for key, content in output_content.items():
                if isinstance(content, dict):
                    try:
                        if content[""] == None:
                            del content[""]
                        else:
                            print(
                                "ERROR: There is an empty account with non-null value at {} when updating {} for period {}"
                                .format(_id, key, output_content['Period']))
                            continue
                    except:
                        pass

                    ### Generate the ES output
                    output_.append({"timestamp":
                                    {"startdate": start,
                                        "enddate": end
                                    },
                                    "period": output_content['Period'],
                                    "reporttype": key,
                                    "data": content
                                    })
        else:
            continue
    return output_

In [13]:
get_fad_acc("CSTC", "Valuation ratios", j[1]["Valuation ratios"][0], LOOKUP_DICT, MAPPING_DICT)

'trailing_EPS'

In [14]:
o = output("CSTC", j, LOOKUP_DICT, MAPPING_DICT)
o

{'2014201404201406': {'ID': '4',
  'Period': 'Quarter 2-2014',
  'Valuation ratios': {'trailing_EPS': 37.0,
   'BVPS': 10815.0,
   'P_TO_E': 668.84,
   'P_TO_B': 0.43,
   'P_TO_S': 6.58,
   'div_yield': 0.0,
   'BETA': 0.77,
   'EV_TO_EBIT': 124.18,
   'EV_TO_EBITDA': 105.87},
  'Profitability ratios': {'gross_profit_margin': 7.47,
   'EBIT_margin': 5.78,
   'EBIDA_TO_net_revenue': 6.78,
   'net_profit_margin': 0.98,
   'ROE': 0.06,
   'ROCE': 0.37,
   'ROA': 0.05},
  'Growth rates': {'net_revenue_growth': 73.13,
   'gross_profit_growth': 54.03,
   'profit_before_tax_growth': -44.92,
   'profit_after_tax_parent_shareholders_growth': -44.34,
   'total_assets_growth': 7.43,
   'lt_liabilities_growth': 0.0,
   'liabilities_growth': 30.57,
   'owners_equity_growth': 0.2,
   'charter_capital_growth': 0.0},
  'Liquidity ratios': {'cash_ratio': 0.17,
   'quick_ratio': 0.74,
   'quick_ratio_except_inventories_st_ar': 0.17,
   'current_ratio': 0.85,
   'interest_coverage': 1.3},
  'Efficiency r

In [15]:
processFinanceInfoX(o, _id="")

[{'timestamp': {'startdate': 1396310400000, 'enddate': 1404086400000},
  'period': 'Quarter 2-2014',
  'reporttype': 'Valuation ratios',
  'data': {'trailing_EPS': 37.0,
   'BVPS': 10815.0,
   'P_TO_E': 668.84,
   'P_TO_B': 0.43,
   'P_TO_S': 6.58,
   'div_yield': 0.0,
   'BETA': 0.77,
   'EV_TO_EBIT': 124.18,
   'EV_TO_EBITDA': 105.87}},
 {'timestamp': {'startdate': 1396310400000, 'enddate': 1404086400000},
  'period': 'Quarter 2-2014',
  'reporttype': 'Profitability ratios',
  'data': {'gross_profit_margin': 7.47,
   'EBIT_margin': 5.78,
   'EBIDA_TO_net_revenue': 6.78,
   'net_profit_margin': 0.98,
   'ROE': 0.06,
   'ROCE': 0.37,
   'ROA': 0.05}},
 {'timestamp': {'startdate': 1396310400000, 'enddate': 1404086400000},
  'period': 'Quarter 2-2014',
  'reporttype': 'Growth rates',
  'data': {'net_revenue_growth': 73.13,
   'gross_profit_growth': 54.03,
   'profit_before_tax_growth': -44.92,
   'profit_after_tax_parent_shareholders_growth': -44.34,
   'total_assets_growth': 7.43,
   'l